In [2]:
# Install dependencies
%pip install llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp guidance transformers[torch]
# %pip install llama-cpp-python
%PATH="/usr/local/cuda-12.3/bin:$PATH" CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --force-reinstall --no-cache llama-cpp-python # For CUDA support



  Obtaining dependency information for llama-index-embeddings-huggingface from https://files.pythonhosted.org/packages/d5/ee/db2cf547e2f40e801a839ef9bbd5a2f6cc74894b5ae8bdfc21cb8dde273c/llama_index_embeddings_huggingface-0.1.4-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%PATH="/usr/local/cuda-12.3/bin:$PATH"` not found.


In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

# Initialize the LLM (e.g., LlamaCPP with your model)
llm = LlamaCPP(
    model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.0,
    max_new_tokens=2048,
    context_window=4096,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
)

# Initialize the embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")



# Load documents
documents = SimpleDirectoryReader("./data", recursive=True).load_data()

# Create an index from the documents using the service context
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Now you can use the index to perform queries
query_engine = index.as_query_engine(llm=llm, similarity_top_k=10,  chunk_size=256, chunk_overlap=64)


/home/daniel/code/llamaindex-rag-glossary/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                   

In [3]:
response = query_engine.query("Who is the developer of spotlight?")
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     387.18 ms
llama_print_timings:      sample time =       1.51 ms /    14 runs   (    0.11 ms per token,  9265.39 tokens per second)
llama_print_timings: prompt eval time =    1774.88 ms /  2164 tokens (    0.82 ms per token,  1219.23 tokens per second)
llama_print_timings:        eval time =     253.28 ms /    13 runs   (   19.48 ms per token,    51.33 tokens per second)
llama_print_timings:       total time =    2055.48 ms /  2177 tokens


 The developers of Spotlight are the team at Renumics.
